# MINI PROJECT FP GROWTH

In [17]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from mlxtend.frequent_patterns import fpgrowth, association_rules

filepath = 'courses_recommender/ui/data/user_with_courses.csv'
data = pd.read_csv(filepath)

In [18]:
#Prepare data for FP-Growth
transactions = data['item'].str.split(', ').tolist()
all_items = sorted(set(item for transaction in transactions for item in transaction))

# Create one-hot encoding for transactions
encoded_data = pd.DataFrame(
    [[1 if item in transaction else 0 for item in all_items] for transaction in transactions],
    columns=all_items
).astype(bool)

In [63]:
#Train FP-Growth model
min_support = 0.02
frequent_itemsets = fpgrowth(encoded_data, min_support=min_support, use_colnames=True)
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=.6)
rules.to_csv('courses_recommender/ui/data/fp_growth_association_rules.csv', index=False)
# print(rules)

In [64]:
input_courses = {'DS0101EN', 'DA0101EN'}

recommended = set()

for _, rule in rules.iterrows():
    if all(course in input_courses for course in rule['antecedents']):
        recommended.update(rule['consequents'])
        print('updated')

recommendations = recommended - input_courses

if recommendations:
    print("Recommended courses:", recommendations)
else:
    print("No recommendations")
# {'PY0101EN', 'BD0211EN', 'DS0103EN', 'ML0115EN', 'BD0111EN', 'DV0101EN', 'BD0101EN', 'ML0101ENv3', 'DS0105EN'}

updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
Recommended courses: {'PY0101EN', 'DS0103EN', 'DS0105EN', 'DV0101EN'}
